# Clone the repo

In [1]:
!git clone https://github.com/wingatesv/CUDA-PointPillars.git

Cloning into 'CUDA-PointPillars'...
remote: Enumerating objects: 418, done.
remote: Counting objects: 100% (203/203), done.
remote: Compressing objects: 100% (110/110), done.
remote: Total 418 (delta 128), reused 119 (delta 92), pack-reused 215
Receiving objects: 100% (418/418), 79.95 MiB | 31.88 MiB/s, done.
Resolving deltas: 100% (158/158), done.


# Setup the OpenPCDet

In [2]:
!sh /content/CUDA-PointPillars/setup_openpcdet.sh

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.9/75.9 MB 9.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 71.5/71.5 kB 10.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 227.7/227.7 kB 30.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 88.3/88.3 kB 13.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 25.5/25.5 MB 12.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 307.2/307.2 kB 36.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 111.6/111.6 kB 16.6 MB/s eta 0:00:00
  Created wheel for fire: filename=fire-0.5.0-py2.py3-none-any.whl size=116934 sha256=f8a658d17c961ae752125d1cea7d04aa4fc94846e62dc134abd129a1df6da293
  Stored in directory: /root/.cache/pip/wheels/90/d4/f7/9404e5db0116bd4d43e5666eaa3e70ab53723e1e3ea40c9a95
Successfully built fire
Cloning into 'OpenPCDet'...
remote: Enumerating objects: 3967, done.
remote: Counting objects: 100% (483/

# Edit the BASE_CONFIG path of the pointpillar.yaml

In [3]:
%%writefile /content/OpenPCDet/tools/cfgs/custom_models/pointpillar.yaml

CLASS_NAMES: ['pass','fail']

DATA_CONFIG:
    _BASE_CONFIG_: /content/OpenPCDet/tools/cfgs/dataset_configs/custom_dataset.yaml
    POINT_CLOUD_RANGE: [-1.82, -0.41, 0.06, 30.18, 31.59, 40.72]
    DATA_PROCESSOR:
        - NAME: mask_points_and_boxes_outside_range
          REMOVE_OUTSIDE_BOXES: True

        - NAME: shuffle_points
          SHUFFLE_ENABLED: {
            'train': True,
            'test': False
          }

        - NAME: transform_points_to_voxels
          VOXEL_SIZE: [0.2, 0.2, 40.66]
          MAX_POINTS_PER_VOXEL: 32
          MAX_NUMBER_OF_VOXELS: {
            'train': 40000,
            'test': 40000
          }
    DATA_AUGMENTOR:
        DISABLE_AUG_LIST: ['placeholder', 'gt_sampling', 'random_world_rotation', 'random_world_scaling']
        AUG_CONFIG_LIST:

            - NAME: random_world_flip
              ALONG_AXIS_LIST: ['x']


MODEL:
    NAME: PointPillar

    VFE:
        NAME: PillarVFE
        WITH_DISTANCE: False
        USE_ABSLOTE_XYZ: True
        USE_NORM: True
        NUM_FILTERS: [64]

    MAP_TO_BEV:
        NAME: PointPillarScatter
        NUM_BEV_FEATURES: 64

    BACKBONE_2D:
        NAME: BaseBEVBackbone
        LAYER_NUMS: [3, 5, 5]
        LAYER_STRIDES: [2, 2, 2]
        NUM_FILTERS: [64, 128, 256]
        UPSAMPLE_STRIDES: [1, 2, 4]
        NUM_UPSAMPLE_FILTERS: [128, 128, 128]

    DENSE_HEAD:
        NAME: AnchorHeadSingle
        CLASS_AGNOSTIC: False

        USE_DIRECTION_CLASSIFIER: True
        DIR_OFFSET: 0.78539
        DIR_LIMIT_OFFSET: 0.0
        NUM_DIR_BINS: 2

        ANCHOR_GENERATOR_CONFIG: [
            {
                'class_name': 'pass',
                'anchor_sizes': [[0.8, 0.6, 1.73]],
                'anchor_rotations': [0, 1.57],
                'anchor_bottom_heights': [-0.6],
                'align_center': False,
                'feature_map_stride': 2,
                'matched_threshold': 0.5,
                'unmatched_threshold': 0.35
            },
            {
                'class_name': 'fail',
                'anchor_sizes': [[0.8, 0.6, 1.73]],
                'anchor_rotations': [0, 1.57],
                'anchor_bottom_heights': [-0.6],
                'align_center': False,
                'feature_map_stride': 2,
                'matched_threshold': 0.5,
                'unmatched_threshold': 0.35
            }
        ]

        TARGET_ASSIGNER_CONFIG:
            NAME: AxisAlignedTargetAssigner
            POS_FRACTION: -1.0
            SAMPLE_SIZE: 512
            NORM_BY_NUM_EXAMPLES: False
            MATCH_HEIGHT: False
            BOX_CODER: ResidualCoder

        LOSS_CONFIG:
            LOSS_WEIGHTS: {
                'cls_weight': 2.0,
                'loc_weight': 4.0, #2.0,
                'dir_weight': 1.0, #0.2,
                'code_weights': [2.0, 2.0, 2.0, 1.0, 1.0, 1.0, 1.0]
            }

    POST_PROCESSING:
        RECALL_THRESH_LIST: [0.3, 0.5, 0.7]
        SCORE_THRESH: 0.1 #0.25
        OUTPUT_RAW_SCORE: False

        EVAL_METRIC: kitti

        NMS_CONFIG:
            MULTI_CLASSES_NMS: False
            NMS_TYPE: nms_gpu
            NMS_THRESH: 0.01
            NMS_PRE_MAXSIZE: 4096
            NMS_POST_MAXSIZE: 500


OPTIMIZATION:
    BATCH_SIZE_PER_GPU: 4
    NUM_EPOCHS: 80

    OPTIMIZER: adam_onecycle
    LR: 0.003 #0.0003
    WEIGHT_DECAY: 0.01
    MOMENTUM: 0.9

    MOMS: [0.95, 0.85]
    PCT_START: 0.4
    DIV_FACTOR: 10
    DECAY_STEP_LIST: [35, 45, 65]
    LR_DECAY: 0.1
    LR_CLIP: 0.0000001

    LR_WARMUP: True
    WARMUP_EPOCH: 1

    GRAD_NORM_CLIP: 10


Overwriting /content/OpenPCDet/tools/cfgs/custom_models/pointpillar.yaml


# Export Custom ONNX Model

In [4]:
!python /content/CUDA-PointPillars/tool/exporter.py --ckpt /content/CUDA-PointPillars/tool/checkpoint_epoch_100.pth --cfg_file /content/OpenPCDet/tools/cfgs/custom_models/pointpillar.yaml

Installing onnxruntime by `/usr/bin/python3 -m pip install onnxruntime`, please wait for a moment..
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.2/6.2 MB 18.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.0/46.0 kB 6.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.8/86.8 kB 11.3 MB/s eta 0:00:00
fatal: not a git repository (or any of the parent directories): .git
2023-10-30 10:58:44,035   INFO  ------ Convert OpenPCDet model for TensorRT ------
/usr/local/lib/python3.10/dist-packages/torch/functional.py:504: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at ../aten/src/ATen/native/TensorShape.cpp:3526.)
  return _VF.meshgrid(tensors, **kwargs)  # type: ignore[attr-defined]
2023-10-30 10:58:44,602   INFO  ==> Loading parameters from checkpoint /content/CUDA-PointPillars/tool/checkpoint_epoch_100.pth to CPU
2023-10-30 10:58:44,653   INFO  ==> Checkpoint trained from v

# Move Custom ONNX into model_custom

In [5]:
!mv /content/pointpillar.onnx /content/CUDA-PointPillars/model_custom/
!mv /content/params.h /content/CUDA-PointPillars/model_custom/

# Install CUDA and TensorRT

In [6]:
!sh /content/CUDA-PointPillars/install_cuda_tensorrt.sh

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
Note, selecting 'nvidia-docker2' for glob 'nvidia*'
Note, selecting 'nvidia-cuda-toolkit-doc' for glob 'nvidia*'
Note, selecting 'nvidia-dkms-450-server' for glob 'nvidia*'
Note, selecting 'nvidia-headless-390' for glob 'nvidia*'
Note, selecting 'nvidia-cuda-toolkit-gcc' for glob 'nvidia*'
Note, selecting 'nvidia-headless-418' for glob 'nvidia*'
Note, selecting 'nvidia-headless-430' for glob 'nvidia*'
Note, selecting 'nvidia-headless-435' for glob 'nvidia*'
Note, selecting 'nvidia-headless-440' for glob 'nvidia*'
Note, selecting 'nvidia-headless-450' for glob 'nvidia*'
Note, selecting 'nvidia-headless-455' for glob 'nvidia*'
Note, selecting 'nvidia-headless-460' for glob 'nvidia*'
Note, selecting 'nvidia-headless-465' for glob 'nvidia*'
Note, selecting 'nvidia-headless-470' for glob 'nvidia*'
Note, selecting 'nvidia-dkms-470-server' for glob 'nvidia*'
Note, selecting 'nvidia-headless-495' f

# Run Main.cpp with demo model

In [7]:
# Change directory to CUDA-PointPillars
%cd CUDA-PointPillars
# Remove cache file
!rm model/pointpillar.onnx.cache
# Run CMake and make commands
!cmake CMakeLists.txt && make -j$(nproc)
# Run the demo
!./demo --data /content/CUDA-PointPillars/data/ --save-dir /content/CUDA-PointPillars/eval/kitti/object/pred_velo/ --model /content/CUDA-PointPillars/model/pointpillar.onnx
# Change directory back to the parent directory (optional)
%cd ../

/content/CUDA-PointPillars
rm: cannot remove 'model/pointpillar.onnx.cache': No such file or directory
CMake Deprecation Warning at CMakeLists.txt:16 (cmake_minimum_required):
  Compatibility with CMake < 3.5 will be removed from a future version of
  CMake.

  Update the VERSION argument <min> value or use a ...<max> suffix to tell
  CMake that the project does not need compatibility with older versions.


-- The C compiler identification is GNU 11.4.0
-- The CXX compiler identification is GNU 11.4.0
-- Detecting C compiler ABI info
-- Detecting C compiler ABI info - done
-- Check for working C compiler: /usr/bin/cc - skipped
-- Detecting C compile features
-- Detecting C compile features - done
-- Detecting CXX compiler ABI info
-- Detecting CXX compiler ABI info - done
-- Check for working CXX compiler: /usr/bin/c++ - skipped
-- Detecting CXX compile features
-- Detecting CXX compile features - done
CMake Warning (dev) at CMakeLists.txt:18 (find_package):
  Policy CMP0146 is not set

# Change param.h with custom param.h

In [8]:
!mv /content/CUDA-PointPillars/include/params.h /content/CUDA-PointPillars/model/
!mv /content/CUDA-PointPillars/model_custom/params.h /content/CUDA-PointPillars/include/

# Run Main.cpp with Custom Model

In [13]:
# Change directory to CUDA-PointPillars
%cd CUDA-PointPillars
# Remove cache file
!rm model_custom/pointpillar.onnx.cache
# Run CMake and make commands
!cmake CMakeLists.txt && make -j$(nproc)
# Run the demo
!./demo --data /content/CUDA-PointPillars/custom_data/ --save-dir /content/CUDA-PointPillars/eval/custom/ --model /content/CUDA-PointPillars/model_custom/pointpillar.onnx
# Change directory back to the parent directory (optional)
%cd ../

/content/CUDA-PointPillars
CMake Deprecation Warning at CMakeLists.txt:16 (cmake_minimum_required):
  Compatibility with CMake < 3.5 will be removed from a future version of
  CMake.

  Update the VERSION argument <min> value or use a ...<max> suffix to tell
  CMake that the project does not need compatibility with older versions.


CMake Warning (dev) at CMakeLists.txt:18 (find_package):
  Policy CMP0146 is not set: The FindCUDA module is removed.  Run "cmake
  --help-policy CMP0146" for policy details.  Use the cmake_policy command to
  set the policy and suppress this warning.

This warning is for project developers.  Use -Wno-dev to suppress it.

-- Configuring done (0.0s)
-- Generating done (0.0s)
-- Build files have been written to: /content/CUDA-PointPillars
[ 11%] Building NVCC (Device) object CMakeFiles/demo.dir/src/demo_generated_preprocess_kernels.cu.o
[ 22%] Building NVCC (Device) object CMakeFiles/demo.dir/src/demo_generated_pillarScatterKernels.cu.o
[ 33%] Building NVCC (

In [ ]:
!python /content/CUDA-PointPillars/visualize_point_cloud.py  --bin_file /content/CUDA-PointPillars/custom_data/000000.bin --pred_file /content/CUDA-PointPillars/eval/custom/000000.txt